In [2]:
import mysql.connector
import pandas as pd
import json

In [3]:
conn = mysql.connector.connect(
         host='10.3.2.13',
         database='etl',
         user='etlmysql',
         password='tata@123')

In [16]:
feedname = 'Employees'
mapper = 'SELECT * FROM etl.mapper' + ' WHERE feedname' + " ='" + feedname + "' and key_col = 'layoutfilepath'"
pd_df = pd.read_sql(mapper, conn)
mapper = spark.createDataFrame(pd_df)


In [18]:
layoutfilepath = mapper.collect()[0]['value_col']
print(layoutfilepath)

/home/etl/ETL/inputdata/EmployeesSchema.json


In [13]:
import sys
import logging
import json
import os, time  #import statements
from datetime import datetime
import mysql.connector
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from stat import *
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.types import StringType
import pyspark.sql.functions as f

In [14]:
SparkContext.setSystemProperty("hive.metastore.uris", "thrift://10.3.2.20:9083")
sparkSession = (SparkSession.builder.appName('pyspark-to-load-tables-hive').enableHiveSupport().getOrCreate())
spark = SparkSession.builder.appName('changeColNames').getOrCreate()
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sqlContext = SQLContext(sc)

In [8]:
data = spark.read.json("/home/etl/ETL/mysqlDetails.json")

In [35]:
data.show()

+-------+---------+--------+--------------------+--------+
|DB_name|     host|password|               query|    user|
+-------+---------+--------+--------------------+--------+
|    etl|10.3.2.13|tata@123|select * from tut...|etlmysql|
+-------+---------+--------+--------------------+--------+



In [9]:
data = pd.read_json('mysqlDetails.json',typ='series',orient='columns')

ValueError: Expected object or value

In [37]:
data

DB_name                             etl
host                          10.3.2.13
user                           etlmysql
password                       tata@123
query       select * from tutorials_tbl
dtype: object

In [42]:
DB_name= data.DB_name
user=data.user
password=data.password
host=data.host
query=data.query
conn = mysql.connector.connect(
     host=host,
     database=DB_name,
     user=user,
     password=password)
#attendence tracker table 
pd_df = pd.read_sql(query, conn)
#dataframe = spark.createDataFrame(pd_df)
pd_df

,tutorial_id,tutorial_title,tutorial_author,submission_date
0,1,Learn PHP,John Poul,2019-07-08
1,2,Learn MySQL,Abdul S,2019-07-08
2,3,JAVA Tutorial,Sanjay,2007-05-06


In [40]:
dataframe = spark.createDataFrame(pd_df)

In [41]:
dataframe.show()

Py4JJavaError: An error occurred while calling o254.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:452)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:588)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:571)
	at o

In [3]:
pd_df = pd.read_sql('select * from  etl.filemetadata', conn)
pd_df

,FeedName,FileName,FileType,FileDelimiter,BusDatePosition,BusDateStartPos,BusDateEndPos,BusDateFormat,TrailerCountPos,HeaderIdentifier,DetailIdentifier,TrailerIdentifier
0,Bank,CreditCard_Nulls,DELIMITED,",",3.0,None,None,DD/MM/YYYY,2.0,1.0,2.0,3.0
1,Reports,Report_US_Alerts_DUMP_LAST_3_M,DELIMITED,",",NaN,None,None,None,NaN,NaN,NaN,NaN


In [4]:
pd_df = pd.read_sql('select * from  etl.feedcontrol', conn)
pd_df

,FeedName,FileName,FileFormat,LandingPath,RejectThreshold,Processtype,HeaderTrailerFlag,RawZonePath,SourceTableName
0,Bank,CreditCard_Nulls.csv,DELIMITED,/home/etl/ETL/inputdata,3,ETL,Y,hdfs://10.3.2.13:8020/user/had,CreditCard_hivetb
1,Reports,Report_US_Alerts_Dump_Last_3_M,DELIMITED,/home/etl/ETL/inputdata,4,passthrough,N,hdfs://10.3.2.13:8020/user/had,ReportUSAlerts_DumpLast3Months
2,Tutorial,MysqlTestTable,database,"""""",0,passthrough,N,hdfs://10.3.2.13:8020/user/had,MysqlTestTable_hivetb


In [5]:
pd_df = pd.read_sql('select * from  etl.calender', conn)
pd_df

,BusDate,OpenIndicator
0,None,Y


In [1]:
import json
import mysql.connector
import pyarrow as pa
import pyarrow.parquet as pq
from stat import *
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.types import StringType
import pyspark.sql.functions as f

SparkContext.setSystemProperty("hive.metastore.uris", "thrift://10.3.2.20:9083")
sparkSession = (SparkSession.builder.appName('pyspark-to-load-tables-hive').enableHiveSupport().getOrCreate())
spark = SparkSession.builder.appName('changeColNames').getOrCreate()
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

# # 1.Data

# In[2]:
BussinessDate = '2019-05-29'#sys.argv[1]''
Tracking_ID = 9 #sys.argv[1]''
#selectSql = 'select * from etl.telcocustomerchurn where busdate =' + BussinessDate +' and attendence_id=' + Tracking_ID;

selectSql = 'select * from etl.telcocustomerchurn where busdate="2019-05-29" and attendence_id=9'
telcom = sparkSession.sql(selectSql)
telcom.toPandas()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn,busdate,attendence_id
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,Month-to-month,Yes,Electronic check,29,29.0,No,2019-05-29,9
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,One year,No,Mailed check,56,1889.0,No,2019-05-29,9
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,53,108.0,Yes,2019-05-29,9
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,No,One year,No,Bank transfer (automatic),42,1840.0,No,2019-05-29,9
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,Month-to-month,Yes,Electronic check,70,151.0,Yes,2019-05-29,9
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,Yes,Month-to-month,Yes,Electronic check,99,820.0,Yes,2019-05-29,9
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,Yes,No,Month-to-month,Yes,Credit card (automatic),89,1949.0,No,2019-05-29,9
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,Month-to-month,No,Mailed check,29,301.0,No,2019-05-29,9
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Month-to-month,Yes,Electronic check,104,3046.0,Yes,2019-05-29,9
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,One year,No,Bank transfer (automatic),56,3487.0,No,2019-05-29,9


In [2]:
type(telcom)

pyspark.sql.dataframe.DataFrame

In [4]:
import pandas as pd
telcom = pd.read_csv("/home/etl/ETL/inputdata/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [5]:
type(telcom)

pandas.core.frame.DataFrame